In [2]:
import pandas as pd
import requests
import shutil
import datetime
import ddddocr
from IPython.display import Image
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

session = requests.Session()

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
}


In [3]:
signal = 1
while signal:
    tmp = round(datetime.datetime.now().timestamp()*1000)
    res = session.get(f'https://portal.sw.nat.gov.tw/APGA/Captcha_validNumGenerate?time={tmp}', stream = True, verify = False)
    f = open('check.png', 'wb')
    shutil.copyfileobj(res.raw, f)
    f.close()

    ocr = ddddocr.DdddOcr()
    with open('check.png', 'rb') as f:
        img_bytes = f.read()
    code = ocr.classification(img_bytes)

    '''防呆code'''
    if len(code) == 6:
        signal = 0
        for char in code:
            if ord(char) > 57:
                signal = 1
                break

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i


In [4]:
payload = {
    'checkCaptcha': code,
    'language': 'chinese'
}

In [5]:
res = session.post('https://portal.sw.nat.gov.tw/APGA/Captcha_checkCaptchaCode', data = payload)

In [22]:
def get_data(data_type):
    if data_type == 'import': 
        TypePort = '3'
    else:
        TypePort ='6'

    # payload = {'pStartyear': '113',
    # 'pEndyer': '113',
    # 'pStartmonth': '1',
    # 'pEndmonth': '8',
    # 'pGtNote':'',
    # 'pColSeq': '進出口別 / 日期',
    # 'pCnyList': '',
    # 'minYear': '92',
    # 'maxYear': '113',
    # 'maxMonth': '8',
    # 'minMonth': '1',
    # 'maxYearByYear': '113',
    # 'searchInfo.TypePort': TypePort,
    # 'searchInfo.TypeTime': '0',
    # 'searchInfo.StartYear': '92',
    # 'searchInfo.StartMonth': '1',
    # 'searchInfo.EndYear': '113',
    # 'searchInfo.EndMonth': '8',
    # 'searchInfo.goodsType': '0',
    # 'searchInfo.goodsName': '21',
    # 'searchInfo.goodsCodeGroup': '8207502000',
    # 'searchInfo.groupType': '1',
    # 'searchInfo.CountryName':'',
    # 'searchInfo.Type': '1',
    # 'searchInfo.OrderType': '進出口別 / 日期',
    # 'searchInfo.ExportType': 'web',
    # 'searchInfo.Captcha': code}

    payload = {'pStartyear': '92',
        'pEndyer': '113',
        'pStartmonth': '1',
        'pEndmonth': '1',
        'pGtNote': '',
        'pColSeq': '進出口別 / 日期 / 貨品別 / 國家(地區)別',
        'pCnyList': '中國大陸',
        'minYear': '92',
        'maxYear': '113',
        'maxMonth': '8',
        'minMonth': '1',
        'maxYearByYear': '113',
        'searchInfo.TypePort': TypePort,
        'searchInfo.TypeTime': '0',
        'searchInfo.StartYear': '92',
        'searchInfo.StartMonth': '1',
        'searchInfo.EndYear': '113',
        'searchInfo.EndMonth': '1',
        'searchInfo.goodsName': '21',
        'searchInfo.goodsType': '2',
        'searchInfo.goodsCodeGroup': '8207602000',
        'searchInfo.groupType': '2',
        'searchInfo.CountryName': '中國大陸',
        'searchInfo.Type': '1',
        'searchInfo.OrderType': '進出口別 / 日期 / 貨品別 / 國家(地區)別',
        'searchInfo.ExportType': 'web',
        'searchInfo.Captcha': code
    }

    res = session.post('https://portal.sw.nat.gov.tw/APGA/GA30_LIST', data = payload,  verify = True, headers = headers)
    soup = BeautifulSoup(res.text, "html.parser")
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(res.content, 'lxml')

    # Find the table that contains the monthly import/export data
    table = soup.find('table', {'id': 'dataList'})

    # Create a list to hold the data
    data = []

    # Iterate through the rows of the table and extract data
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        # print(columns)
        month_data = {
            "date": columns[1].text.strip().split('\n')[0],
            "國家": columns[5].text,
            f"{data_type}_美元": columns[6].text.strip()
        }
        data.append(month_data)

    return pd.DataFrame(data)

In [23]:
data_import = get_data('import')
data_export = get_data('export')

In [24]:
data_import

,date,國家,import_美元
0,92年6月,中國大陸,0
1,92年7月,中國大陸,0
2,92年9月,中國大陸,0
3,92年10月,中國大陸,0
4,93年1月,中國大陸,0
...,...,...,...
215,112年9月,中國大陸,8
216,112年10月,中國大陸,3
217,112年11月,中國大陸,20
218,112年12月,中國大陸,5


In [26]:
pd.merge(data_import, data_export, how = 'outer')

,date,國家,import_美元,export_美元
0,100年10月,中國大陸,2,323
1,100年11月,中國大陸,NaN,330
2,100年12月,中國大陸,NaN,522
3,100年1月,中國大陸,17,363
4,100年2月,中國大陸,NaN,308
...,...,...,...,...
241,99年5月,中國大陸,37,345
242,99年6月,中國大陸,21,283
243,99年7月,中國大陸,9,213
244,99年8月,中國大陸,23,363


In [8]:
data_import

,category,date,國家,usd_amount
0,進口總值(含復進口),92年6月,中國大陸,0
1,進口總值(含復進口),92年7月,中國大陸,0
2,進口總值(含復進口),92年9月,中國大陸,0
3,進口總值(含復進口),92年10月,中國大陸,0
4,進口總值(含復進口),93年1月,中國大陸,0
...,...,...,...,...
215,進口總值(含復進口),112年9月,中國大陸,8
216,進口總值(含復進口),112年10月,中國大陸,3
217,進口總值(含復進口),112年11月,中國大陸,20
218,進口總值(含復進口),112年12月,中國大陸,5


In [14]:
data_export

,category,date,國家,usd_amount
0,出口總值(含復出口),92年3月,中國大陸,0
1,出口總值(含復出口),92年4月,中國大陸,0
2,出口總值(含復出口),92年9月,中國大陸,1
3,出口總值(含復出口),92年10月,中國大陸,2
4,出口總值(含復出口),92年11月,中國大陸,16
...,...,...,...,...
221,出口總值(含復出口),112年9月,中國大陸,553
222,出口總值(含復出口),112年10月,中國大陸,552
223,出口總值(含復出口),112年11月,中國大陸,882
224,出口總值(含復出口),112年12月,中國大陸,868


In [9]:
test = pd.concat([data_import, data_export])

In [10]:
test.to_excel('1_進出口總值(美元).xlsx')

# 視覺化

In [11]:
df = pd.read_excel('1_進出口總值(美元).xlsx')

In [12]:
x = df[df['category'] == '出口總值(含復出口)']['date'].values
y = df[df['category'] == '出口總值(含復出口)']['usd_amount'].values